<center><h1>Py-Galaxies examples</h1></center>

# SUB-HALO Stars

## Import modules

In [11]:
import PyGalaxies
import time
import numpy as np


model_param_filepath='Input_Params/input_params.yml'
debug_flag = False
verbosity = 1 
time_code = True

## Get model params and print

In [12]:
model_params = PyGalaxies.ModelParams(model_param_filepath,verbosity,debug_flag)
model_params.output_params() 

input_files         : {'graph_file': '../Input Data/mega_graph_mini_09-11.hdf5'}
output_files        : {'halo_file': '../Output Data/SMT13HaloOutput.hdf5', 'galaxy_file': '../Output Data/SMT13GalaxyOutput.hdf5'}
cosmology           : {'omega_m': {'Description': 'Matter density parameter', 'Value': 0.3, 'Units': 'None'}, 'f_baryon': {'Description': 'Baryon fraction', 'Value': 0.155, 'Units': 'None'}}
model_switches      : {'HOD': {'Description': 'Halo occupation description for stars (as primitive test)', 'Value': True}, 'sub_halo': {'Description': 'For each graph, load in Sub-halo data?', 'Value': True}}
performance         : {'io_nRec': {'Description': 'Size of HDF5 io buffer (number of records)', 'Value': 1000}}


## Run main routine

In [13]:
# Open the input HDF5 file containing graph groups and create an output file.
HDF_properties = PyGalaxies.HDFProperties(model_params)



start_time = time.time()


for graph_ID in range(0,HDF_properties.no_of_graphs)[:]:

    if verbosity >= 2: print('Processing graph',graph_ID)
    

    graph_properties = PyGalaxies.GraphProperties(graph_ID,
                                       HDF_properties.graph_input_file,
                                       model_params,
                                       HDF_properties.part_mass)
    
    
    array_of_halo_properties = np.empty(len(graph_properties.graph_halo_ids),
                                        dtype = object)
    
    if type(graph_properties.n_subhalos) == int:
        continue
    
    for snap_ID in graph_properties.generation_id:
        

        if graph_properties.generation_start_index[snap_ID] == 2**30:
            continue
    
        
        this_generation_halo = graph_properties.graph_halo_ids[
                               graph_properties.generation_start_index[snap_ID]:
                               graph_properties.generation_start_index[snap_ID] + 
                               graph_properties.generation_length[snap_ID]]
        
        for halo_ID in this_generation_halo:
            
            halo = PyGalaxies.HaloProperties(str(graph_ID), snap_ID, halo_ID,
                                  graph_properties, HDF_properties.part_mass,
                                  HDF_properties.dtype_subhalo_stores)
        
        
            PyGalaxies.inclusive_mass_contribution(halo)
            
            if halo.nprog > 0: 
                PyGalaxies.gather_progenitors(halo,HDF_properties.part_mass)
            else:
                halo.mass_from_progenitors = 0.
            
            PyGalaxies.set_baryon_fraction(halo, array_of_halo_properties,
                                           model_params.f_baryon)
            
            
            

             
            halo.collect_galaxy_prog_info(graph_properties, HDF_properties.part_mass,
                                          array_of_halo_properties)
            
            
            
            
                
       
            
        
            halo.done=True
            
            array_of_halo_properties[halo_ID] = halo

            HDF_properties.n_halo +=1 

    # Save output
    HDF_properties.output_halos(array_of_halo_properties)




# Close input file
HDF_properties.close_graph_io(HDF_properties.graph_input_file)


# Close output files
if HDF_properties.halo_output_iRec > 0: 

    halo_output_iRec = HDF_properties.flush_output()
    
HDF_properties.close_graph_io(HDF_properties.halo_output_file)


end_time = time.time()
print('This took {} seconds'.format(end_time-start_time))

This took 35.53764581680298 seconds


In [14]:
vars(array_of_halo_properties[-1])

{'graph_ID': '975',
 'snap_ID': 61,
 'halo_ID': 22,
 'catalog_ID': 1816,
 'mass': 39903177160.09238,
 'nprog': 1,
 'prog_start': 21,
 'prog_end': 22,
 'prog_ids': array([21]),
 'prog_mass': array([3.99031772e+10]),
 'ndesc': -1,
 'desc_start': 1073741824,
 'desc_end': 1073741823,
 'desc_ids': array([], dtype=int32),
 'desc_mass': array([], dtype=float64),
 'mean_pos': array([33.91094678, 50.83798803, 36.65987155]),
 'velocity_dispersion_3D': 37.06783550241392,
 'mean_vel': array([-305.18989563,   60.86648401,  142.69911067]),
 'rms_radius': 0.044236223452654815,
 'mass_baryon': 6184992459.814319,
 'mass_from_progenitors': 5.307545158232393e+19,
 'mass_baryon_from_progenitors': 0.0,
 'inclusive_contribution': array([], dtype=float64),
 'done': True,
 'subhalo_start_index': 9,
 'n_subhalo': 1,
 'sub_halo_attrs': array([(9, 3.89219978e+19, 10.)],
       dtype=[('sub_graph_ids', '<i4'), ('prog_DM_mass', '<f8'), ('stellar_mass', '<f8')]),
 'sub_graph_halo_ids': array([9]),
 'central_galaxy_

In [15]:
dtype = np.dtype([
                ('sub_graph_ids', np.int32),
                ('prog_DM_mass', np.float64),
                ('stellar_mass', np.float64),
                ('descended', bool)
                ])

In [ ]:

test_array[:]['sub_graph_ids'] = (1,2,34,5,6)
print(test_array)
gal_id = 2

In [ ]:
np.where((test_array['sub_graph_ids'] == gal_id))

In [61]:
test_array = np.empty(100, dtype = dtype)
print(test_array)

[(         0,  1.17711000e+005, 4.51560706e-311, False)
 ( -16777216, -3.24180904e+178, 1.40808709e-321, False)
 (         0,  6.01347019e-154, 1.05117818e-153,  True)
 ( 925969969,  1.21089429e-099, 2.40585681e-057,  True)
 ( 926037808,  8.04882561e-096, 1.03234313e-259,  True)
 ( 825040936,  1.03896569e-095, 5.05576908e-038,  True)
 (1697919033,  3.64464666e-086, 2.00702056e-052,  True)
 ( 825373485,  2.85427980e-109, 6.01347002e-154,  True)
 ( 807411744,  2.90252661e-057, 1.26931768e-076,  True)
 ( 824192054,  9.34863709e-067, 1.05192079e-153,  True)
 (1702195796,  4.76640645e-038, 1.05118733e-153,  True)
 ( 926035507,  1.21179510e-099, 2.90737394e-033,  True)
 ( 926300473,  8.52611571e-096, 1.03234438e-259,  True)
 ( 892936232,  1.17959814e-095, 7.11876656e-067,  True)
 (1698182197,  4.24787606e-086, 5.64151042e-062,  True)
 ( 808988971,  2.85533142e-109, 2.63100583e-052,  True)
 ( 875902256,  5.06149091e-038, 1.26931635e-076,  True)
 ( 908078134,  5.64150309e-062, 1.05162661e-153,

In [7]:
x = False

In [9]:
if not x:
    print('hey')

hey
